# Optimizing Event Locations and Urban Management in Melbourne

**Authored by:** Sahan Chamod

---

**Duration:** 90 mins  
**Level:** Intermediate  
**Pre-requisite Skills:** Python, Data Analysis, Pandas, Data Visualization


## Scenario:
The city of Melbourne hosts numerous events annually, including weddings, film shoots, public events, promotions, and fun runs. Optimal event site selection is critical to ensure attendee convenience, minimal disruption, and efficient urban space utilization. Currently, event planners and city authorities often face challenges with inadequate location suitability data, resulting in pedestrian congestion, parking difficulties, and unintended urban disruptions. This project aims to solve these issues by providing data-driven insights into optimal event location selection.

## Project Objective:
To identify optimal locations for different types of events in Melbourne by analyzing historical event permit data, pedestrian traffic trends, and on-street parking availability.

## Project Stages:

### Stage 1: Data Collection & Integration
- Acquire datasets from:
  - Event Permits (2014-2018)
  - Pedestrian Counting System (hourly counts)
  - On-Street Parking Bays
- Cleanse and preprocess datasets to handle missing values, anomalies, and integrate data into a unified schema suitable for analysis.

### Stage 2: Exploratory Data Analysis (EDA)
- Analyze event permit data to identify frequent event locations, types, and durations.
- Explore pedestrian count data to identify hourly, daily, weekly, and seasonal traffic patterns.
- Assess parking bay data to evaluate the availability and accessibility of parking spaces near potential event sites.

### Stage 3: Location Suitability Analysis
- Develop analytical models integrating event frequency, pedestrian traffic intensity, and parking availability to rate locations.
- Classify locations based on event suitability, accessibility, and expected disruption.
- Identify peak traffic and parking constraint periods to avoid scheduling major disruptive events.

### Stage 4: Visualization and Dashboard Development
- Create an interactive dashboard using visualization tools (e.g., Power BI, Tableau) to present suitability scores, historical event distribution, pedestrian traffic trends, and parking availability.
- Provide city planners, event organizers, and local businesses easy access to insights for informed decision-making.

### Stage 5: Validation and Recommendations
- Validate the model by comparing predicted optimal locations with historical successful events.
- Offer recommendations for infrastructure improvements and targeted policy-making to support high-demand areas for large-scale events.

## Expected Outcomes:
- Enhanced efficiency in selecting and managing event locations.
- Improved attendee experience through reduced congestion and better parking management.
- Reduced disruptions in pedestrian and vehicular traffic.
- Informed infrastructure development aligned with city event management needs.

## Dataset Descriptions:

### 1. Event Permits 2014-2018
- Contains permits data for events such as film shoots, photo shoots, weddings, Christmas parties, promotions, fun runs, and public events.
- Useful for understanding event frequency, type, duration, and spatial distribution across Melbourne.

### 2. Pedestrian Counting System (Hourly Counts)
- Records pedestrian activity hourly from sensors placed at various city locations.
- Essential for identifying pedestrian traffic patterns, peak periods, and location-specific pedestrian volumes.

### 3. On-Street Parking Bays
- Consists of spatial polygons representing on-street parking bays across Melbourne.
- Provides data on parking locations, availability, and restrictions, critical for assessing parking accessibility near potential event locations.